In [ ]:
import numpy as np
import matplotlib.pylab as plt
import sys
sys.path.insert(0, '../zdrojaky/kf')
from trajectory import trajectory
np.set_printoptions(precision=2)

# Třída pro Kalmanův filtr

Můžete vykopírovat a importovat ze souboru pro snazší ladění.

In [ ]:
class KF():
    def __init__(self, A, B, H, R, Q):
        self.A = A
        self.B = B
        self.H = H
        self.R = R
        self.Q = Q
        self.P = *** matice s 1000 na diagonále
        self.x = *** nulový vektor
        self.log_x = []
    
    def predict(self, u=None):
        xminus = ***
        if u is not None:
            xminus += self.B.dot(u)
        Pminus = ***
        self.x = xminus
        self.P = Pminus
    
    def correct(self, yt):
        HPHT = ***
        zavorka_inv = ***
        K = ***
        innovation = ***
        xplus = self.x + K.dot(innovation)
        zavorka = ***
        KRKT = ***
        Pplus = zavorka.dot(self.P).dot(zavorka) + KRKT
        self.x = xplus
        self.P = Pplus
    
    def log(self):
        self.log_x.append(self.x)

# Cvičení 4: Sledování objektu ve 2D

Cílem tohoto cvičení je filtrace polohy objektu pohybujícího se na ploše, tj. ve 2D. Cílem je odhadovat stavový vektor sestávající se z polohy v osách $x_1$ a $x_2$ a rychlostech v obou těchto osách.

$$
x_t =
\begin{bmatrix}
x_{1,t} \\ 
x_{2,t} \\ 
v_{x_1,t} \\ 
v_{x_2,t}
\end{bmatrix}
$$

Víme, že pro polohu v první ose platí

$$
x_{1,t} = x_{1,t-1} + v_{x_1,t} dt + w_{x_1,t},
$$

a že analogicky platí totéž i pro polohu v ose druhé. Abychom se nemuseli zaobírat zrychlením, budeme uvažovat, že rychlost je stejná a její změny jsou způsobeny jen šumem,

$$
v_{x_1,t} = v_{x_1, t-1} + w_{vx_1, t}.
$$

Analogicky pro druhou složku rychlosti. K dispozici jsou jen měření polohy v obou osách a to v časovém kroku 0.1s.

Potřebujeme tedy sestavit stavový model

$$
\begin{align}
x_t &= A_t x_{t-1} + B_t u_t + w_t, &w_t \sim (0, Q_t),\\
y_t &= H_t x_t + \varepsilon_t, &\varepsilon_t \sim (0, R_t).
\end{align}
$$

**Úkol 1: Sestavte stavový model a určete matice**

Kovarianční matice nám musí někdo dodat, níže jsou uvedeny.

\begin{align*}
    A &= ***
    \quad
    &Q &= q\cdot
    \begin{bmatrix}
        \frac{dt^3}{3}    & 0                 & \frac{dt^{2}}{2}  & 0  \\
        0                 & \frac{dt^3}{3}    & 0                 & \frac{dt^{2}}{2} \\
        \frac{dt^{2}}{2}  & 0                 & dt                & 0 \\
        0                 & \frac{dt^{2}}{2}  & 0                 & dt
    \end{bmatrix}
    \notag \\
    H &= ***
    \quad
    &R &=
    r^{2}\cdot
    \begin{bmatrix}
        1 & 0 \\
        0 & 1
    \end{bmatrix}
\end{align*}
kde $dt = 1$, $q = .5$, $r=3$.

**Úkol 2: dodefinujte proměnné**

In [ ]:
q = ***
dt = ***
r = ***
A = ***
Q = q * np.array([[dt**3/3, 0      , dt**2/2, 0      ],
                  [0,       dt**3/3, 0,       dt**2/2],
                  [dt**2/2, 0,       dt,      0      ],
                  [0,       dt**2/2, 0,       dt     ]])
H = ***
R = ***

**Úkol 3: doplňte za `code` libovolné číslo (slouží jako seed generátoru). V objektu `traj` budete mít trajektorii - atribut `Y` - o 100 řádcích, v každém je poloha v obou osách. Vykreslete ji křížky.**

In [ ]:
code = ***
traj = trajectory(code)
plt.figure(figsize=(10, 10))
plt.plot(***)
plt.axis('equal')
plt.show()

Kalmanův filtr je napsán jako třída KF. Tu si jednoduše pustíme, provede výpočty za vás. Předávají se jí jednotlivá měření polohy, tj. řádky v `traj.Y.T`

In [ ]:
kf = KF(***)
for yt in traj.Y.T:
    kf.predict()
    kf.correct(yt)
    kf.log()

log_x = np.array(kf.log_x).T      # Zde najdeme odhad

**Úkol 4: `log_x` má 4 řádky, v každém sloupci je jeden odhad stavu $x_{t}$. Vykreslete stejně jako výše naměřenou polohu a její filtrovaný odhad**

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(***, '.', label='Měření')
plt.plot(***, '-', color='red', label='Filtrovaný odhad')
plt.plot(traj.X[0,:], traj.X[1,:], 'k', label='Skutečnost')
plt.grid(True)
plt.axis('equal')
plt.legend()
plt.show()

**Úkol 5: V odhadech najdete i odhady rychlosti. Vykreslete je jako 2 subploty vedle sebe.**

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1,2,1)
plt.plot(***, 'r')
plt.subplot(1,2,2)
plt.plot(***)
plt.show()

**Úkol 6: `traj` má atribut X, tedy `traj.X`, což je matice o 4 řádcích, obsahující skutečné hodnoty polohy a rychlosti, které jsme se snažili Kalmanovým filtrem odhadnout. Doplňte je do obrázků černě.**

**Zbyde-li čas, zkuste:**

- vykreslit rezidua odhadu
- vynechat pár měření a postupovat jen s predikcí, nikoliv korekcí
- použít nesprávné kovariance šumu
- odstartovat ze špatného iniciálního odhadu stavu.